In [4]:
import pandas as pd

df = pd.read_csv("Datos TP - Listado Electivas 5to.csv", sep=",")

print(df)

    Código                                           Electiva   Última  \
0    82045                                     Bioinformática  2023 2C   
1    82010  Desarrollo de Aplicaciones para Dispositivos M...  2023 2C   
2    82113  Economía del Conocimiento\n ex: 082082 - Marke...  2023 2C   
3    82110  Gestión de la Transformación Organizacional\n ...  2023 2C   
4    82080                           Inteligencia de Negocios  2023 2C   
5    82084                           Modelos Organizacionales  2023 2C   
6    82044                     Tecnologías avanzadas en Redes  2023 2C   
7    82014        Tecnologías aplicadas a soluciones de Datos  2023 2C   
8    82111  Plataformas de Virtualización\n ex: 082043 - I...  2023 2C   
9    82121  Base de Datos No SQL\n ex: 082042 - Implementa...  2023 2C   
10   82118  Ciudades Digitales y Gobierno Electronico\n ex...  2023 2C   
11   82105  Desarrollo de Modelos Analiticos (machine lear...  2023 2C   
12   82115  Dirección Estratégica\n ex